In [1]:
# Bibliotecas
import re
import unicodedata
import numpy as np
import pandas as pd
from datetime import datetime
from funcoes.utils import converter_string_numero

2024-03-05 17:58:00,441 [logging.log_init] INFO: LOGLEVEL=INFO


# Funções

# Execução

In [2]:
# Carregar dados brutos
df = pd.read_csv(f'dados/1_EXT/fii_raw_fundamentus_' + datetime.today().strftime('%Y-%m-%d') + '.csv')

In [3]:
# Fazer tratamentos iniciais
df.columns = [c.upper().replace('?','').strip() for c in df.columns]
# Fazer tratamentos adicionais
df.columns = [re.sub('\s+', ' ', c) for c in df.columns]
df.columns = [c.replace('. ', '_') for c in df.columns]
df.columns = [c.replace(' / ', '_POR_') for c in df.columns]
df.columns = [c.replace('/', '_POR_') for c in df.columns]
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('(', '') for c in df.columns]
df.columns = [c.replace(')', '') for c in df.columns]
df.columns = [unicodedata.normalize('NFD', c).encode('ascii', 'ignore').decode("utf-8")  for c in df.columns]
df.columns = [re.sub('_+', '_', c) for c in df.columns]

In [4]:
# Especial colunas para ficar
cols = ['FII','NOME', 'MANDATO', 'SEGMENTO', 'GESTAO', 'ATIVOS', 'QTD_IMOVEIS', 'QTD_UNIDADES',    # Informações gerais
        'COTACAO', 'DATA_ULT_COT', 'MIN_52_SEM', 'MAX_52_SEM', 'VOL_$_MED_2M',                     # Valor de cotas
        'VALOR_DE_MERCADO', 'NRO_COTAS',                                                           # Market
        'DIA', 'MES', '30_DIAS', '12_MESES', '2023', '2022', '2021', '2020', '2019',               # Oscilações
        'RECEITA_12_MESES','VENDA_DE_ATIVOS_12_MESES', 'FFO_12_MESES', 'REND_DISTRIBUIDO_12_MESES',# Indicadores
        'RECEITA_3_MESES', 'VENDA_DE_ATIVOS_3_MESES','FFO_3_MESES', 'REND_DISTRIBUIDO_3_MESES',    # Indicadores
        'FFO_POR_COTA', 'FFO_YIELD', 'DIV_YIELD', 'P_POR_VP', 'VP_POR_COTA', 'DIVIDENDO_POR_COTA', # Indicadores
        'PATRIM_LIQUIDO', 'IMOVEIS_POR_PL_DO_FII', 'AREA_M2', 'ALUGUEL_POR_M2', 'PRECO_DO_M2',     # Indicadores
        'CAP_RATE', 'VACANCIA_MEDIA']                                                              # Indicadores

In [5]:
# Selecionar
df = df[cols].copy()

In [6]:
# Renomear
df.rename(columns={'DIA': 'OSCILACOES_%_DIA',
                   'MES': 'OSCILACOES_%_MES',
                   '30_DIAS': 'OSCILACOES_%_30_DIAS',
                   '12_MESES': 'OSCILACOES_%_12_MESES',
                   '2023': 'OSCILACOES_%_2023',
                   '2022': 'OSCILACOES_%_2022',
                   '2021': 'OSCILACOES_%_2021',
                   '2020': 'OSCILACOES_%_2020',
                   '2019': 'OSCILACOES_%_2019'}, inplace=True)

In [7]:
# Colunas de atributos
cols_atr = ['FII','NOME', 'MANDATO', 'SEGMENTO', 'GESTAO', 'DATA_ULT_COT']
cols_num = [c for c in df.columns if c not in cols_atr] 

In [8]:
# Criar backup para comparar
df_bkp = df.copy()

In [9]:
# Converter colunas numericas
for cn in cols_num:
    df[cn] = df[cn].apply(lambda x:converter_string_numero(x))

In [10]:
# Checar tipos
df[cols_num].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ATIVOS                     341 non-null    int64  
 1   QTD_IMOVEIS                341 non-null    int64  
 2   QTD_UNIDADES               337 non-null    float64
 3   COTACAO                    341 non-null    float64
 4   MIN_52_SEM                 341 non-null    float64
 5   MAX_52_SEM                 341 non-null    float64
 6   VOL_$_MED_2M               341 non-null    int64  
 7   VALOR_DE_MERCADO           341 non-null    int64  
 8   NRO_COTAS                  341 non-null    int64  
 9   OSCILACOES_%_DIA           341 non-null    float64
 10  OSCILACOES_%_MES           341 non-null    float64
 11  OSCILACOES_%_30_DIAS       341 non-null    float64
 12  OSCILACOES_%_12_MESES      341 non-null    float64
 13  OSCILACOES_%_2023          340 non-null    float64

In [11]:
# Salvar
df.to_csv(f'dados/2_TRA/fii_tratado_fundamentus_' + datetime.today().strftime('%Y-%m-%d') + '.csv', index=False)